In [ ]:
import pickle
from collections.abc import Callable
from pprint import pprint

import numpy as np
import pandas as pd

In [ ]:
with open("../artifacts/human_eval_cache.pkl", "rb") as f:
    cache = pickle.load(f)

len(cache)

In [ ]:
keys = list(cache.keys())
values = list(cache.values())
value_keys = list(values[0].keys())
value_values = list(values[0].values())

print(
    f"Key: {type(keys[0])}",
    f"Key elements: {[type(k) for k in keys[0]]}",
    f"Value: {type(values[0])}",
    f"Value key: {type(value_keys[0])}",
    f"Value value: {type(value_values[0])}",
    sep="\n",
)

In [ ]:
print("Key:")
pprint(keys[0])
print()
print("Value:")
pprint(values[0])

In [ ]:
hashes = [hash(t) for t in cache]
len(hashes), len(set(hashes))

In [ ]:
df = pd.DataFrame(
    [
        {
            "q": q,
            "a": a,
            "n": n,
            "ann": sorted(ann["val_annotations"]),
        }
        for (q, a, n), ann in cache.items()
    ],
)

In [ ]:
df.head()

In [ ]:
df["ann"] = df["ann"].map(lambda x: [i + 2 for i in x])
df["ann"].explode().agg(["min", "max"])

In [ ]:
df["std"] = df["ann"].map(np.std)
df["std"].describe()

In [ ]:
df.sort_values("std")

In [ ]:
df[df["ann"].map(lambda x: x == [0, 2, 4])]

In [ ]:
lst = [0, 2, 4]
np.diff(lst)

In [ ]:
def listeq(lst: list[int]) -> Callable[[list[int]], bool]:
    def eq(el: list[int]) -> bool:
        return lst == el

    return eq


def entropy(data: list[int]) -> float:
    counts = np.bincount(data)
    p = counts / len(data)
    return -np.sum(p[p > 0] * np.log2(p[p > 0]))


def gini_coefficient(data: list[int]) -> float:
    if np.all(data == data[0]) or np.isclose(np.sum(data), 0):
        return 0

    sorted_data = np.sort(data)
    n = len(data)

    return (
        2 * np.sum(np.arange(1, n + 1) * sorted_data) / (n * np.sum(sorted_data))
    ) - (n + 1) / n


def calculate_alpha(ratings: list[int]) -> float:
    """
    Calculate Krippendorff's Alpha for a single example with nominal data.

    :param ratings: List of ratings (integers or strings) for a single example by different raters.
    :return: Krippendorff's Alpha as a float.
    """
    arratings = np.array(ratings)

    # Count the occurrences of each rating
    _, counts = np.unique(arratings, return_counts=True)
    n = len(arratings)

    # Calculate observed disagreement (D_o)
    D_o = sum(c * (c - 1) for c in counts)  # Pairwise comparisons for each category
    D_o = 0 if n <= 1 else 1 - D_o / (n * (n - 1))

    # Calculate expected disagreement (D_e)
    D_e = 1 - sum((counts / n) ** 2)  # Prob. of random agreement for each category

    # Calculate Krippendorff's Alpha
    return 1 - D_o / D_e if D_e != 0 else 1  # Handle division by zero


dd = df.assign(
    entropy=lambda x: x["ann"].map(entropy),
    gini=lambda x: x["ann"].map(gini_coefficient),
    alpha=lambda x: x["ann"].map(calculate_alpha),
)
examples = [
    [0, 0, 0],
    [1, 1, 1],
    [1, 2, 3],
    [0, 0, 4],
    [0, 2, 4],
]
pd.concat(
    [dd[dd["ann"].map(listeq(lst))].iloc[0] for lst in examples],
    axis=1,
).transpose()[["ann", "std", "entropy", "gini", "alpha"]]

In [ ]:
dd.drop_duplicates("ann")[["ann", "entropy", "gini", "alpha"]].sort_values(
    "alpha", ascending=False
)